# STA410 Coding Challenge II (10 points)

Welcome.

- If you experience technical issues while working on this coding challenge, send a message to sta410@utoronto.ca.

- Any live messages that need to be communicated to the class in real-time during the coding challenge will be posted on [piazza](https://piazza.com/utoronto.ca/winter2023/sta410).

- ***You may add cells for scratch work***, but if required answers are not submitted through the provided cells where the answers are requested your answers will not be graded.

- ***If you accidentally delete a required cell***, try "Edit > Undo Delete Cells" in the notebook editor; otherwise, redownload the notebook (so it has the correct required cells ids) and repopulate it with your answers (assuming you don't overwrite them).




## Rules

0. **This is an individual assignment.** You are not permitted to engage in any form of active or passive collusion or collaboration with other human beings while working on this challenge.  ***You may NOT access chat or communication applications during the coding challenge.***


1. You are welcome and encouraged to adapt code you find available online into your notebook; however, if you do so you must provide a link to the utilized resource. ***Failure to list such references may result in a loss of points.***


2. **Do not delete or replace cells**: this erases `cell ids` upon which automated code tests are based.

  > ***If you are working in any environment other than*** [UofT JupyterLab](https://jupyter.utoronto.ca/hub/user-redirect/git-pull?repo=https://github.com/pointOfive/sta410hw0&branch=master&urlpath=/lab/tree/sta410hw0), [UofT JupyterHub](https://jupyter.utoronto.ca/hub/user-redirect/git-pull?repo=https://github.com/pointOfive/sta410hw0&branch=master), or [Google Colab](https://colab.research.google.com/github/pointOfive/sta410hw0/blob/master/sta410hw0.ipynb), your system must meet the following versioning requirements 
   >
   >   - [notebook format >=4.5](https://github.com/jupyterlab/jupyterlab/issues/9729) 
   >   - jupyter [notebook](https://jupyter.org/install#jupyter-notebook) version [>=6.2](https://jupyter-notebook.readthedocs.io/en/stable/) for "classic" notebooks served by [jupyterhub](https://jupyterhub.readthedocs.io/en/stable/quickstart.html)
   >   - [jupyterlab](https://jupyter.org/install) version [>=3.0.13](https://github.com/jupyterlab/jupyterlab/releases/tag/v3.0.13) for "jupyterlab" notebooks  
   >    
   > otherwise `cell ids` will not be supported and you will not get any credit for your submitted homework.  
      
      
3. **No cells may have any runtime errors** because this causes subsequent automated code tests to fail and you will not get marks for tests which fail because of previous runtime errors. 

  - Run time errors include, e.g., unassigned variables, mismatched parentheses, and any code which does not work when the notebook cells are sequentially run, even if it was provided for you as part of the starter code. ***It is best to restart and re-run the cells in your notebook to ensure there are no runtime errors before submitting your work.***
    
  - The `try`-`except` block syntax catches runtime errors and transforms them into `exceptions` which will not cause subsequent automated code tests to fail.  


4. **No jupyter shortcut commands** such as `! pwd` or `%%timeit` may be included in the final submission as they will cause subsequent automated code tests to fail.

  - ***Comment out ALL jupyter shortcut commands***, e.g., `# ! pwd` or `# %%timeit` in submitted notebooks.

5. **Python library imports are limited** to only libraries imported in the starter code and the [standard python modules](https://docs.python.org/3/py-modindex.html). Importing additional libraries will cause subsequent automated code tests to fail.

  > Unless a problem instructs differently, you may use any functions available from the libraries imported in the starter code; otherwise, you are expected to create your own Python functionality based on the Python stdlib (standard libary, i.e., base Python and standard Python modules).

In [82]:
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Problem 1 (5 points)

This problem will explore ***Iteratively Reweighted Least Squares IRLS*** in the context of a ***log linear model*** with a ***Poisson*** distribution. 

For a ***log-linear model*** $\lambda_i = E[y_i|x_i, \beta] = Var[y_i|x_i, \beta] = \exp(x^T_i\beta)$ with a ***Poisson*** distribution

$$f\left(y_i|\lambda_i= \exp(x^T_i\beta)\right) = \frac{\exp(-\lambda_i)\lambda^y_i}{y_i!} = \frac{\exp(-\exp(x_i^T\beta))\exp(x_i^T\beta)^{y_i}}{y_i!}$$

For this ***generalized linear model***
- the ***link function*** $\phi$ is $\phi(\lambda_i) = \ln \lambda_i = x_i^T\beta$ so $\lambda_i = E[y_i|x_i, \beta] = Var[y_i|x_i, \beta] = \exp(x_i^T\beta)$
    - and for the $\phi(\cdot) = \ln(\cdot)$ ***link function***, $\phi'(\cdot) = (\cdot)^{-1}$ 
- the ***log likelihood*** ignoring the $(y_i!)$ ***normalizing constant*** is $$\ln f(y_i|\lambda_i) = -\lambda_i + y_i\ln\lambda_i = -\exp(x_i^T\beta) + y_i (x_i^T\beta)$$ 
- the ***score function*** is $$S(\beta) = \sum_{i=1}^n \nabla_\beta \ln f(y_i|\lambda_i) = \sum_{i=1}^n  -\exp(x_i^T\beta)x_i + y_i x_i = \sum_{i=1}^n (y_i-\lambda_i)x_i = X^T(y-\lambda)$$ 
- and the ***expected Fisher Information*** is 

  \begin{align*}
  \mathcal I(\beta) = -E\left[\sum_{i=1}^n H \ln f(y_i|\lambda_i)\right] & = {} -E\left[\sum_{i=1}^n J \nabla_\beta \ln f(y_i|\lambda_i)\right] \\
  & = {} E\left[\sum_{i=1}^n (\nabla_\beta \ln f(y_i|\lambda_i))(\nabla_\beta \ln f(y_i|\lambda_i))^T\right] \\
  & = {} \sum_{i=1}^n E\left[(y_i-\lambda_i)^2\right]x_ix_i^T = \sum_{i=1}^n  \lambda_ix_ix_i^T  = X^TD(\lambda)X \quad D_{ii}(\lambda)=\lambda_i \quad D_{\underset{\neq}{ij}}(\lambda)=0
  \end{align*}


***Newton's method*** and ***Fisher scoring*** for this specification are thus, respectively

\begin{align*}
\beta^{(t+1)} & = {} \beta^{(t)} - [H f(y|\lambda^{(t)})]^{-1} S(\beta^{(t)})\\
\beta^{(t+1)} & = {} \beta^{(t)} + \mathcal I(\beta)^{-1} S(\beta^{(t)})\\
& = {} \beta^{(t)} + (X^TD(\lambda^{(t)})X)^{-1} S(\beta^{(t)})\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} \left(X^TD(\lambda^{(t)})X\beta^{(t)} + S(\beta^{(t)}) \right)\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} \Bigg(X^TD(\lambda^{(t)})X\beta^{(t)} + X^TD(\lambda^{(t)})[D(\lambda^{(t)})]^{-1} \left(y-\lambda^{(t)}\right) \Bigg)\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} X^TD(\lambda^{(t)}) \Bigg(X\beta^{(t)} + \overbrace{\left[\frac{y-\lambda^{(t)}}{\lambda^{(t)}} \right]}^{\text{element-wise}} \Bigg)\\
\end{align*}

which is the solution of the ***weighted least squares regression*** of $\tilde y^{(t)} = \left(X\beta^{(t)} + \frac{y-\lambda^{(t)}}{\lambda^{(t)}}\right)$ against $X$ with ***weights*** $D(\lambda^{(t)})$.

## Problem 1 Questions 0-1 (1 point)

0. (0.5 points) Which of the following are equal to $D(\lambda^{(t)})$?

    1. $\hat y^{(t)}$ 
    2. $E[y | x_i,\lambda^{(k)}]$
    3. $Var[y | x_i,\lambda^{(k)}]$
    4. All of the above  
    

1. (0.5 points) Which of the following are equal to $\tilde y^{(t)}$? 

    1. $\hat y^{(t)} + \frac{y - E[y | x_i,\lambda^{(t)}]}{\hat y^{(t)}}$ 
    2. $\hat y^{(t)} + \frac{y - E[y | x_i,\lambda^{(t)}]}{SD[y | x_i,\lambda^{(t)}]}$ 
    3. $\ln \hat y^{(t)} + \frac{y - \hat  y^{(t)}}{Var[y | x_i,\lambda^{(t)}]}$
    4. $\ln \hat y^{(t)} + \frac{y - E[y | x_i,\lambda^{(t)}]}{SD[y | x_i,\lambda^{(t)}]}$ 
    
    
***Hint:*** The $\hat y^{(t)}$ is the prediction of $y$ at time $t$. What is that?


In [83]:
# 0.5 points each [format: `str` either "A" or "B" or "C" or "D" based on the choices above]
p1q0 = "D" 
p1q1 = "A"
# Uncomment the above and keep each only either "A" or "B" or "C" or "D"

# This cell will produce a runtime error until the `p1q0` and `p1q1` variables are assigned values

## Problem 0 Question 2-3 (1 point)

2. (0.5 points) For the design matrix `X` below, and $\beta_j = \frac{10-j}{10}$ with intercept $\beta_0$, produce $E[y|X,\beta]$ for the ***Poisson GLM*** specified above.

    ```python
    np.random.seed(410)
    n,p=100,10
    X = stats.norm.rvs(size=(n,p))
    X[:,0]=1
    X
    ```

2. (0.5 points) Now simulate $y$ for the ***Poisson GLM*** specified above with seed `np.random.seed(411)`. That is, call `np.random.seed(411)` immediately before sampling $y$.


In [84]:
# Cell for scratch work
np.random.seed(410)
n,p=100,10
X = stats.norm.rvs(size=(n,p))
X[:,0]=1
X

beta = np.array([(10 - j) / 10 for j in range(p)])

E_y = np.exp(X @ beta)
print(E_y)
# You are welcome to add as many new cells into this notebook as you would like.
# Just do not leave in a state that will produce a runtime errors when notebook cells are run sequentially.

# Any cells included for scratch work that are no longer needed may be deleted so long as 
# - all the required functions are still defined and available when called
# - no cells requiring variable assignments are deleted.
np.random.seed(411)
y = np.random.poisson(E_y)
# None of this will not cause problems with `cell ids` assuming your versioning supports `cell ids`
# (as UofT JupyterHub, UofT JupyterLab, an Google Colab will).


[1.45426013e+01 8.77226326e+00 2.25583642e+00 2.04507248e+00
 7.97666158e+00 1.20874614e+00 1.19472636e+00 6.36253244e-02
 2.20642523e+00 1.61003896e+02 9.34320756e+00 2.36846712e+01
 7.79104068e-01 9.94085696e-01 5.29664021e+00 2.96087194e+00
 2.93570231e+00 1.00567142e-01 2.91684950e+00 7.74493967e+00
 4.74856275e+00 3.81288206e+00 7.23325901e+00 2.88742605e-01
 1.10084585e+00 7.61625320e-01 4.25337252e-01 1.06479211e+00
 3.62642622e+00 7.85917080e-01 1.94186196e-01 4.94032667e+01
 9.01175320e-01 7.24928003e+00 2.03533166e+01 3.04811941e+00
 4.17791168e+00 6.50250521e+00 1.35476393e+00 2.04976550e-01
 5.91533774e+00 2.84849171e-01 7.56678181e+00 2.56207016e+00
 3.51272151e+00 1.97881541e+00 1.42973765e+01 1.36727808e+00
 6.13828298e+01 4.67900639e+00 2.12069243e+00 6.18475757e-01
 4.45764564e+00 1.09884730e+00 1.05517227e+00 3.29432568e+01
 1.01526417e+00 3.61879551e+00 2.43873191e+01 1.72384846e+00
 2.44829856e-01 1.27622215e+00 7.87179801e-01 3.46550896e+00
 1.85748063e+01 4.615477

In [85]:
# Cell for scratch work


In [86]:
# 0.5 points each [format: `np.array` with shape (100, 1)]
p1q2 = E_y
p1q3 = y # y sampled based on E[y|X,beta] with seed np.random.seed(411)
# Assign the appropriate numpy expressions to `p1q2` and `p1q3`

# This cell will produce a runtime error until the `p1q2` and `p1q3` variables are assigned values

## Problem 1 Question 4 (1 point)

4. Define the function `IRLS(X, y, beta_0, m, v, k)` where the default arguments for parameters `m` and `v` are functions returning `E[y|X,beta_t]` and `Var[y|X,beta_t]`, respectively:

    - `m = lambda X,beta_t: #E[y|X,beta_t]`
    - `v = lambda X,beta_t: #Var[y|X,beta_t]`

  and `IRLS` performs `k` ***IRLS*** updates on `beta_0` using `sm.regression.linear_model.WLS`.
  
Your function will be tested directly for various values of `k` based on `X` and `y` generated above using the initial value `beta_0 = np.zeros((p,1))`.

In [87]:
def IRLS(X, y, beta_0, m = lambda X,beta_t: np.exp(X @ beta_t), v = lambda X,beta_t: np.exp(X @ beta_t), k=1):
    '''
    X (np.array) with shape (n,p)
    y (np.array) with shape (n,1)
    beta_0 (np.array) with shape (p,1)
    m (function) with paramters (np.arrays) X (n,p) and beta_t (p,1) which returns E[y|X,beta_t]
    v (function) with paramters (np.arrays) X (n,p) and beta_t (p,1) which returns Var[y|X,beta_t]
    k (int) number of IRLS steps to perform
    
    returns beta_k (np.array) with shape (p,1) which is the kth IRLS update on beta_0    
    '''
    # Pass the correct functions to m and v 
    
    beta_t = beta_0.copy()
    # <complete>
    for _ in range(k):
        E_y_t = m(X, beta_t)
        Var_y_t = v(X, beta_t)

        z = X @ beta_t + (y.reshape(-1, 1) - E_y_t.reshape(-1, 1)) / Var_y_t.reshape(-1, 1)

        model = sm.regression.linear_model.WLS(z, X, weights=Var_y_t).fit()

        beta_t = model.params.reshape(-1, 1)
    return beta_t

## Problem 1 Question 5-6 (0.5 points)

5. (0.25 points) What is the default function assigned for the `m` parameter of the `IRLS` function above?

6. (0.25 points) What is the default function assigned for the `v` parameter of the `IRLS` function above?

In [88]:
# 0.25 points each [format: functions conforming to the IRLS parameters]

m = lambda X,beta_t: np.exp(X @ beta_t)
p1q5 = m
v = lambda X,beta_t: np.exp(X @ beta_t)
p1q6 = v
# uncomment four lines above once m and v and correctly defined

# This cell will produce a runtime error until the `p1q5` and `p1q6` variables are assigned values

## Problem 1 Question 7-8 (1 point)

These questions consider the ***IRLS*** algorithm from the more general perspective of ***pseudo (log) likelihoods***.

The ***score*** function $S$ and the ***expected Fisher information*** $\mathcal I$ for a ***pseudo (log) likelihood*** $Q$ (in place of an explicit ***log likelihood***) of a ***generalized linear model*** are

\begin{align*}
S_i(\beta) = \nabla_\beta Q(y_i|\lambda_i=\phi^{-1}(x_i^T\beta)) & = {} \frac{\partial}{\lambda_i}Q(y_i|\lambda_i) \nabla_\beta \lambda_i \quad \text{(derivative chain rule)} \\
& = {} \frac{\partial}{\lambda_i}Q(y_i|\lambda_i) \nabla_\beta \phi^{-1}(x_i^T\beta)\\
& = {} \frac{\partial}{\lambda_i}Q(y_i|\lambda_i) [\phi'(\phi^{-1}(x_i^T\beta))]^{-1}x_i \quad \text{(derivative of inverse rule)}\\
& \equiv {} \frac{y_i-\lambda_i}{Var(\lambda_i)} [\phi'(\lambda_i)]^{-1}x_i \quad \text{(pseudo log likelihood specification)}\\\\
S(\beta) & = {} \sum_{i=1}^n \frac{y_i-\lambda_i}{Var(\lambda_i)} [\phi'(\lambda_i)]^{-1}x_i = X^T\underbrace{\left[\frac{y-\lambda}{Var(\lambda)\odot \phi'(\lambda)}\right]}_{\text{element-wise}}\\
\mathcal I_i(\beta) = - E[H Q(y_i|\lambda_i) ] = - E[J \nabla_\beta Q(y_i|\lambda_i) ] & = {} E[ \nabla_\beta Q(y_i|\lambda_i) \nabla_\beta Q(y_i|\lambda_i)^T ] = E[ S(\beta) S(\beta)^T ] \\
& = {} \frac{E[(y_i-\lambda_i)^2]x_ix_i^T}{Var(\lambda_i)^2[\phi'(\lambda_i)]^{2}} = \frac{x_ix_i^T}{Var(\lambda_i)[\phi'(\lambda_i)]^{2}} \\
\mathcal I(\beta) & = {} \sum_{i=1}^n \frac{x_ix_i^T}{Var(\lambda_i)[\phi'(\lambda_i)]^{2}} = X^TD(\lambda)X \quad D_{ii}(\lambda)=\frac{1}{Var(\lambda_i)[\phi'(\lambda_i)]^{2}} \quad D_{\underset{\neq}{ij}}(\lambda)=0
\end{align*}

***Newton's method*** and ***Fisher scoring*** are thus respectively

\begin{align*}
\beta^{(t+1)} & = {} \beta^{(t)} - [H Q(y|\lambda^{(t)})]^{-1} S(\beta^{(t)})\\
\beta^{(t+1)} & = {} \beta^{(t)} + \mathcal I(\beta)^{-1} S(\beta^{(t)})\\
& = {} \beta^{(t)} + (X^TD(\lambda^{(t)})X)^{-1} S(\beta^{(t)})\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} \left(X^TD(\lambda^{(t)})X\beta^{(t)} + S(\beta^{(t)}) \right)\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} \Bigg(X^TD(\lambda^{(t)})X\beta^{(t)} + X^TD(\lambda^{(t)}) \overbrace{\left[\phi'(\lambda^{(t)}) \odot (y-\lambda^{(t)})\right]}^{\text{element-wise}} \Bigg)\\
& = {} (X^TD(\lambda^{(t)})X)^{-1} X^TD(\lambda^{(t)}) \left(X\beta^{(t)} + \left[\phi'(\lambda^{(t)}) \odot (y-\lambda^{(t)}) \right] \right)
\end{align*}

7. (0.5 points) Recalling that $\phi'(\cdot) = (\cdot)^{-1}$ for ***link function*** $\phi(\cdot) = \ln(\cdot)$, use the `IRLS` function defined above to perform ***Fisher scoring*** for `X`, `y`,  initial value `beta_0 = np.zeros((p,1))`, and `k=1000` for the ***quasi (log) likelihood*** based on the ***log linear model*** $\lambda_i = \exp(x_i^T\beta)$ with $Var(\lambda_i) = \exp(x_i^T\beta)$.
    

8. (0.5 points) ***Logistic regression*** with a ***Bernoulli*** distribution

    $$f\left(y_i\,\Bigg|\,\lambda_i= \frac{1}{1+\exp(-x^T_i\beta)}\right) = \lambda_i^{y_i} (1-\lambda_i)^{1-y_i} = \left(\frac{1}{1+\exp(-x^T_i\beta)} \right)^{y_i} \left(1-\frac{1}{1+\exp(-x^T_i\beta)} \right)^{1-y_i} $$

    has 

    - $\lambda_i = E[y_i|x_i,\beta] = \frac{1}{1+\exp(-x^T_i\beta)}$ and $Var[y_i|x_i,\beta] = Var(\lambda_i) = \left(1-\frac{1}{1+\exp(-x^T_i\beta)}\right)\left(\frac{1}{1+\exp(-x^T_i\beta)}\right)$

    - and ***link function*** $\phi(\lambda_i) = \ln\left(\frac{\lambda_i}{1-\lambda_i}\right)$ so that $\phi'(\lambda_i) = \frac{1}{\lambda_i} + \frac{1}{1-\lambda_i} = \frac{1}{\lambda_i(1-\lambda_i)}$.

    Use the `IRLS` function defined above to perform ***Fisher scoring*** for `X`, `(y>1).asype(int)`, initial value `beta_0 = np.zeros((p,1))`, and `k=1000` for the ***quasi (log) likelihood*** based on $\lambda_i = \frac{1}{1+\exp(-x^T_i\beta)}$ with $Var(\lambda_i) = \lambda_i(1-\lambda_i)$.

***Hint:*** these problem will be most easy to solve if you first simplify the element-wise $D(\lambda^{(t)})$ and $\phi'(\lambda^{(t)}) \odot (y-\lambda^{(t)})$ into simpler forms.
   

In [89]:
# 0.5 points each [format: `np.array` with shape (p, 1)]
beta_0 = np.zeros((p,1))
y = y.reshape((-1, 1))

p1q7 = IRLS(X, y, beta_0, m=lambda X, beta_t: np.exp(X @ beta_t), v=lambda X, beta_t: np.exp(X @ beta_t), k=1000) # IRLS(X, y, beta_0, m=..., v=..., k=1000)
p1q8 = IRLS(X, (y>1).astype(int), beta_0, m=lambda X, beta_t: 1/(1 + np.exp(-X @ beta_t)), v=lambda X, beta_t: (1/(1 + np.exp(-X @ beta_t))) * (1 - (1/(1 + np.exp(-X @ beta_t)))), k=1000)
# IRLS(X, (y>1).astype(int), beta_0, m=..., v=..., k=1000)
# Assign the appropriate functions for `m` and `v` in order to correctly assign `p1q7` and `p1q8`

# This cell will produce a runtime error until the `p1q7` and `p1q8` variables are assigned values

## Problem 1 Question 9 (0.5 points)

9. What is the estimated standard error of the coefficients for question 7?

***Hint:*** what is an estimate of the covariance matrix of interest? 

In [90]:
# Cell for scratch work

# You are welcome to add as many new cells into this notebook as you would like.
# Just do not leave in a state that will produce a runtime errors when notebook cells are run sequentially.

# Any cells included for scratch work that are no longer needed may be deleted so long as 
# - all the required functions are still defined and available when called
# - no cells requiring variable assignments are deleted.

# None of this will not cause problems with `cell ids` assuming your versioning supports `cell ids`
# (as UofT JupyterHub, UofT JupyterLab, an Google Colab will).


In [92]:
# Cell for scratch work


[[0.99337894]
 [0.95871935]
 [0.7368231 ]
 [0.67169192]
 [0.52562732]
 [0.52255555]
 [0.3746533 ]
 [0.31115609]
 [0.17089869]
 [0.09678262]]


In [99]:
# 0.5 points [format: `np.array` with shape (p, 1)]

p1q9 = 0#

# This cell will produce a runtime error until the `p1q9` variable has been assigned a value

# Problem 2 (5 points)

Complete ***Problem 2*** in the `STA410_CC2_PyMC.ipynb` file.
- You will use `X` and `y` generated in ***Problem 1***, so recreate that data in the `STA410_CC2_PyMC.ipynb` file.